## **Thiết lập môi trường**

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
! wget -q https://archive.apache.org/dist/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz

In [ ]:
!tar xf spark-3.5.4-bin-hadoop3.tgz

In [ ]:
! pip install -q findspark

In [ ]:
! du -sh spark-3.5.4-bin-hadoop3.tgz

383M	spark-3.5.4-bin-hadoop3.tgz


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.4-bin-hadoop3"

In [ ]:
! echo $JAVA_HOME

/usr/lib/jvm/java-8-openjdk-amd64


In [ ]:
! echo $SPARK_HOME

/content/spark-3.5.4-bin-hadoop3


In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark as spark

print(spark.__version__)

3.5.5


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Các thư viện cần thiết

In [ ]:
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType, StructType, StructField
from pyspark import SparkContext
import itertools

# *PCY Algorithm*
## Định nghĩa lớp PCYAlgorithm để thực hiện thuật toán PCY tìm tập phổ biến.







In [ ]:
class PCYAlgorithm:
    def __init__(self):
        """Initialize the PCY algorithm with default parameters."""
        self.min_support = 0.01
        self.min_confidence = 0.1
        self.basket_column = "Basket"
        self.partition_count = 10
        self.bucket_size = 1000

    def configure_support(self, value: float):
        """Set the minimum support threshold."""
        self.min_support = value
        return self

    def configure_confidence(self, value: float):
        """Set the minimum confidence threshold."""
        self.min_confidence = value
        return self

    def configure_basket_column(self, value: str):
        """Set the name of the column containing baskets."""
        self.basket_column = value
        return self

    def configure_partitions(self, value: int):
        """Set the number of partitions for the DataFrame."""
        self.partition_count = value
        return self

    def configure_buckets(self, value: int):
        """Set the number of buckets for hashing pairs."""
        self.bucket_size = value
        return self

    def process(self, dataframe: DataFrame) -> 'PCYResult':
        """
        Process the input DataFrame using the PCY algorithm.
        Args:
            dataframe: Input DataFrame with a column of baskets (arrays of items).
        Returns:
            PCYResult object containing frequent items, pairs, rules, and basket count.
        """
        sc = SparkContext.getOrCreate()

        # Repartition and count baskets
        repartitioned_df = dataframe.repartition(self.partition_count)
        basket_count = repartitioned_df.count()
        support_threshold = self.min_support * basket_count

        # Step 1: Identify frequent items
        items_df = repartitioned_df.select(F.explode(self.basket_column).alias("item"))
        item_frequencies = items_df.groupBy("item").count()
        frequent_items_df = item_frequencies.filter(F.col("count") >= support_threshold)\
                                          .orderBy(F.col("count").desc())

        # Step 2: Hash pairs into buckets
        pair_structure = ArrayType(StructType([
            StructField("first", StringType()),
            StructField("second", StringType())
        ]))

        def create_pairs(basket):
            return [{"first": x, "second": y} for x, y in itertools.combinations(sorted(basket), 2)]

        pair_udf = F.udf(create_pairs, pair_structure)
        pairs_df = repartitioned_df.withColumn("pair", F.explode(pair_udf(self.basket_column)))
        hashed_pairs_df = pairs_df.withColumn("bucket", F.hash("pair") % self.bucket_size)
        bucket_frequencies = hashed_pairs_df.groupBy("bucket").count()
        frequent_buckets = bucket_frequencies.filter(F.col("count") >= support_threshold)\
                                            .select("bucket").rdd.flatMap(lambda x: x).collect()
        frequent_buckets_set = sc.broadcast(set(frequent_buckets))

        # Step 3: Filter baskets to frequent items
        frequent_items_set = set(frequent_items_df.select("item").rdd.flatMap(lambda x: x).collect())
        broadcast_items = sc.broadcast(frequent_items_set)

        def filter_items(basket):
            return [item for item in basket if item in broadcast_items.value]

        filter_udf = F.udf(filter_items, ArrayType(StringType()))
        filtered_df = repartitioned_df.withColumn("filtered", filter_udf(self.basket_column))

        # Step 4: Generate and count candidate pairs
        candidate_pairs_df = filtered_df.withColumn("pair", F.explode(pair_udf("filtered")))
        hashed_candidates_df = candidate_pairs_df.withColumn("bucket", F.hash("pair") % self.bucket_size)
        filtered_candidates_df = hashed_candidates_df.filter(F.col("bucket").isin(frequent_buckets_set.value))
        pair_frequencies = filtered_candidates_df.groupBy("pair").count()
        frequent_pairs_df = pair_frequencies.filter(F.col("count") >= support_threshold)\
                                         .withColumnRenamed("count", "frequency")\
                                         .orderBy(F.col("frequency").desc())

        # Step 5: Generate association rules
        item_support_df = frequent_items_df.withColumnRenamed("count", "support")
        rules_df = frequent_pairs_df.withColumn("first", F.col("pair.first"))\
                                   .withColumn("second", F.col("pair.second"))
        rules_df = rules_df.join(item_support_df.withColumnRenamed("item", "first")\
                                              .withColumnRenamed("support", "support_first"), "first")
        rules_df = rules_df.join(item_support_df.withColumnRenamed("item", "second")\
                                              .withColumnRenamed("support", "support_second"), "second")
        forward_rules = rules_df.select(
            F.col("first").alias("antecedent"),
            F.col("second").alias("consequent"),
            (F.col("frequency") / F.col("support_first")).alias("confidence")
        )
        reverse_rules = rules_df.select(
            F.col("second").alias("antecedent"),
            F.col("first").alias("consequent"),
            (F.col("frequency") / F.col("support_second")).alias("confidence")
        )
        all_rules_df = forward_rules.union(reverse_rules)\
                                   .filter(F.col("confidence") >= self.min_confidence)\
                                   .orderBy(F.col("confidence").desc())

        return PCYResult(frequent_items_df, frequent_pairs_df, all_rules_df, basket_count)

## Định nghĩa lớp *PCYResult* để lưu và quản lý kết quả từ *PCYAlgorithm*

In [ ]:
class PCYResult:
    def __init__(self, items: DataFrame, pairs: DataFrame, rules: DataFrame, basket_count: int):
        """Initialize the result object with computed counts."""
        self.frequent_items = items.cache()
        self.frequent_pairs = pairs.cache()
        self.association_rules = rules.cache()
        self.basket_count = basket_count
        self.items_count = items.count()
        self.pairs_count = pairs.count()
        self.rules_count = rules.count()

    def retrieve_items(self, limit=20) -> DataFrame:
        """Return DataFrame of frequent items sorted by count in descending order."""
        return self.frequent_items.limit(limit)

    def retrieve_pairs(self, limit=20) -> DataFrame:
        """Return DataFrame of frequent pairs sorted by frequency in descending order."""
        return self.frequent_pairs.limit(limit)

    def retrieve_rules(self, limit=20) -> DataFrame:
        """Return DataFrame of rules sorted by confidence in descending order."""
        return self.association_rules.limit(limit)

    def baskets_total(self) -> int:
        """Return the total number of baskets in the dataset."""
        return self.basket_count

    def items_total(self) -> int:
        """Return the total number of frequent items."""
        return self.items_count

    def pairs_total(self) -> int:
        """Return the total number of frequent pairs."""
        return self.pairs_count

    def rules_total(self) -> int:
        """Return the total number of association rules."""
        return self.rules_count

## Thực thi toàn bộ quy trình từ khởi tạo Spark, tải dữ liệu, chạy thuật toán PCY, đến hiển thị kết quả.

In [ ]:
if __name__ == "__main__":
    # Initialize Spark session
    spark_session = SparkSession.builder.appName("PCY Analysis").getOrCreate()

    # Load and prepare data
    file_path = "/content/drive/MyDrive/Giữa kỳ/baskets.csv"
    raw_df = spark_session.read.csv(file_path, header=True)
    grouped_df = raw_df.groupBy("Member_number", "Date")\
                      .agg(F.collect_set("itemDescription").alias("Basket"))

    # Configure and run PCY algorithm
    pcy_algo = PCYAlgorithm()
    pcy_algo.configure_support(0.01)\
            .configure_confidence(0.1)\
            .configure_basket_column("Basket")\
            .configure_partitions(10)\
            .configure_buckets(1000)
    result = pcy_algo.process(grouped_df)

    # Display results with counts
    print("=== Dataset Summary ===")
    print(f"Total baskets in dataset: {result.baskets_total()}")

    print("\n=== Top Frequent Items (by count) ===")
    print(f"Total frequent items: {result.items_total()}")
    result.retrieve_items().show(truncate=False)

    print("\n=== Top Frequent Pairs (by frequency) ===")
    print(f"Total frequent pairs: {result.pairs_total()}")
    result.retrieve_pairs().show(truncate=False)

    print("\n=== Top Association Rules (by confidence) ===")
    print(f"Total association rules: {result.rules_total()}")
    result.retrieve_rules().show(truncate=False)

    # Clean up
    spark_session.stop()

=== Dataset Summary ===
Total baskets in dataset: 14963

=== Top Frequent Items (by count) ===
Total frequent items: 64
+------------------+-----+
|item              |count|
+------------------+-----+
|whole milk        |2363 |
|other vegetables  |1827 |
|rolls/buns        |1646 |
|soda              |1453 |
|yogurt            |1285 |
|root vegetables   |1041 |
|tropical fruit    |1014 |
|bottled water     |908  |
|sausage           |903  |
|citrus fruit      |795  |
|pastry            |774  |
|pip fruit         |734  |
|shopping bags     |712  |
|canned beer       |702  |
|bottled beer      |678  |
|whipped/sour cream|654  |
|newspapers        |582  |
|frankfurter       |565  |
|brown bread       |563  |
|domestic eggs     |555  |
+------------------+-----+


=== Top Frequent Pairs (by frequency) ===
Total frequent pairs: 5
+------------------------------+---------+
|pair                          |frequency|
+------------------------------+---------+
|{other vegetables, whole milk}|222